# Implement Dropout to Benchmark Ensemble Uncertainties Against

<hr/>

## Prerequisites

#### Imports

In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import tensorflow as tf

from ensemble_uncertainties.model_library import models
from ensemble_uncertainties.evaluators.evaluator_support import format_time_elapsed
from ensemble_uncertainties.evaluators.regression_evaluator import RegressionEvaluator
from ensemble_uncertainties.utils.ad_assessment import rmses_frac

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

#### Constants

In [ ]:
DROPOUT_RATE = .2
PLOT_FOLDER = './plots/different_reps_regression_plots/'

#### Settings

In [ ]:
mpl.rcParams['figure.dpi'] = 200
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(RANDOM_SEED)

#### Functions

In [ ]:
def deep_architecture_dropout(output_activation, dropout_rate=.2):
    """Our default deep architecture for property prediction:

    in | 256 ReLU | 128 ReLU | 16 ReLU | 1
    
    Additionally, dropout is performed in every layer.
        
    Parameters
    ----------
    output_activation : str
        Name of the activation in the final neuron (tf.keras.activations)
            
    Returns
    -------
    function
        A function that takes the number of variables
        (int) and returns the (uncompiled) model
    """
    in_dim = X.shape[1]
    def make_model(n_vars):
        # Create model. To apply dropout during inference,
        # the functional format must be used:
        inp = Input(shape=(in_dim,))
        x = Dropout(dropout_rate)(inp)
        x = Dense(256, activation='relu')(x)
        x = Dropout(dropout_rate)(x, training=True)
        x = Dense(128, activation='relu')(x)
        x = Dropout(dropout_rate)(x, training=True)
        x = Dense(16, activation='relu')(x)
        x = Dropout(dropout_rate)(x, training=True)
        out = Dense(1, activation=output_activation)(x)
        model = Model(inp, out)
        return model
    return make_model


def deep_architecture_dropout(output_activation):
    """Our default deep architecture for property prediction:

    in | 256 ReLU | 128 ReLU | 16 ReLU | 1 
        
    Parameters
    ----------
    output_activation : str
        Name of the activation in the final neuron (tf.keras.activations)
            
    Returns
    -------
    function
        A function that takes the number of variables
        (int) and returns the (uncompiled) model
    """
    def make_model(n_vars):
        model = Sequential()
        model.add(Dense(256, input_shape=(n_vars,), activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(16, activation='relu'))
        model.add(Dense(1, activation=output_activation))
        return model
    return make_model

In [ ]:
# Compile model
model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mse'])